# Register and Rydberg-Atom Interactions

*What you will learn:*
- what is a `Register` and how to create it;
- why the relative position of the atoms in a `Register` is important;
- what is the Rydberg blockade effect;
- how to tune the position of the atoms, depending on the application.

## The `Register`

The `Register` is a group of trapped atoms in a defined (but reconfigurable) configuration. Each atom holds a specific quantum state encoded in specific electronic levels. Usually these are two-level systems and we refer to them as **qubits**.

There are multiple ways to define a `Register`, the most customizable one being to create a dictionary that associates a name (the key) to a cooordinate (the value).

In [ ]:
import numpy as np
import pulser

# Manually generate the coordinates of a 4x4 square with 5μm spacing
L = 4
square = np.array([[i, j] for i in range(L) for j in range(L)], dtype=float)
square -= np.mean(square, axis=0)
square *= 5

qubits = {f"q{i}": coord for i, coord in enumerate(square)}
reg = pulser.Register(qubits)
reg.draw()

If one doesn't particularly care about the name given to the qubits, one can also create a `Register` just from a list of coordinates (using the `Register.from_coordinates` class method). In this case, the qubit ID's are just numbered, starting from 0, in the order they are provided in, with the option of adding a common prefix before each number. Also, it automatically centers the entire array around the origin, an option that can be disabled if desired.

In [ ]:
import pulser

reg2 = pulser.Register.from_coordinates(
    square,
    prefix="q",
)  # All qubit IDs will start with 'q'
print("Qubit IDs:", reg2.qubit_ids)

Furthermore, there are also built-in class methods for creation of common array patterns:

In [ ]:
from pulser import Register

# 3x3 register with 6μm spacing
square_reg = Register.square(side=3, spacing=6.0, prefix="q")
# 4x2 register with 5μm spacing
rect_reg = Register.rectangle(rows=4, columns=2, spacing=5.0, prefix="q")
# 3x2 register with 5μm spacing between rows and 7μm spacing between columns
rect_lattice_reg = Register.rectangular_lattice(
    rows=3, columns=2, row_spacing=5.0, col_spacing=7.0, prefix="q"
)
# 4x3 triangular lattice with 6μm spacing
tri_reg = Register.triangular_lattice(
    rows=4, atoms_per_row=3, spacing=6.0, prefix="q"
)
# An hexagonal cutout of a triangular lattice with 3 layers and 8μm spacing
hex_reg = Register.hexagon(layers=3, spacing=8.0, prefix="q")

For more information on all the `Register` methods, please refer to the [Register API reference](apidoc/core.rst#pulser.register.register.Register).

## Rydberg-Atom Interactions

When excited to a Rydberg state, nearby atoms interact according to the [interaction Hamiltonian](programming.md#interaction-hamiltonian). The interaction strength is always dependent on the distance between atoms and is stronger the closer they are. Therefore, appropriately selecting the atoms' relative positions is a crucial step in the programming of neutral-atom QPUs. 

In the most common case of the [Ising Hamiltonian](programming.md#ising-hamiltonian), the interaction operator is given by

$$
\hat{U}_{ij} = \frac{C_6}{R_{ij}^6} \hat{n}_i \hat{n}_j,
$$

where 

- the interaction strength is $\frac{C_6}{R_{ij}^6}$, with $C_6$ a coefficient that depends on the principal quantum number of the Rydberg state the atoms are excited to;
- the entangling operator between atom $i$ and $j$ is $\hat{n}_i\hat{n}_j = |r\rangle\langle r|_i |r\rangle\langle r|_j$. 

Note that:

1. The interaction strength scales with $R_{ij}^{-6}$, underlying a strong decay with distance.
2. There is only an interaction when both atoms are in their respective Rydberg states, $|r\rangle_i$ and $|r\rangle_j$.

## The Rydberg Blockade 

The *Rydberg blockade effect* is an expression of the interaction between two Rydberg atoms that provides a useful approximation for determining when two atoms are within the interaction range of one another. 

Consider a system of two atoms in the $|gg\rangle$ state that we want to excite to the $|rr\rangle$ state. The interaction Hamiltonian dictates that there is an energy penalty of $\frac{C_6}{R_{ij}^6}$ for being in the $|rr\rangle$ state - that is to say, the energy of the $|rr\rangle$ state is shifted by this amount.

If we try to excite this system to the $|rr\rangle$ state with a resonant pulse ($\delta=0$) of Rabi frequency $\Omega$, the excitation is suppressed when $\hbar\Omega << C_6/R_{ij}$, i.e the energy of the driving term is not sufficient to overcome the extra cost of having the system in in the $|rr\rangle$ state.

Equivalently, this condition is verified when $R_{ij} << R_b$, where

$$R_b = \left(\frac{C_6}{\hbar\Omega}\right)^{(1/6)}$$

is the **Rydberg blockade radius**. When $R_{ij}$ is well within the Rydberg blockade radius, the system is instead excited to $(|gr\rangle + |rg\rangle)/\sqrt{2}$ - notably, an entangle state - with effective Rabi frequency $\sqrt{2}\Omega$. Conversely, the transition to $|rr\rangle$ occurs when the atoms are well *outside* the Rydberg blockade radius.

<center>
    <img src="files/rydberg_blockade.png" alt="Rydberg blockade effect" width="400">
    <figcaption>There is no simultaneous transition to the doubly excited state inside the blockade radius.</figcaption>
</center>

**Important notes**:

- The Rydberg blockade radius is only a useful approximation to reason about whether two atoms interact significantly; it *should not* be interpreted as a discrete threshold beyond which there are no interactions.
- In fact, the approximation is least adequate for values of $R_{ij} \approx R_b$, so placing atoms at distances close to $R_b$ should be done extra carefully.
- Furthermore, $R_b$ depends on the Rabi frequency $\Omega$; as such, **fixing** $R_b$ **also determines** $\Omega$ and vice-versa. 

### Estimating the Rydberg blockade radius

The `Device` class includes methods to calculate the Rydberg blockade radius for a given value of Rabi frequency and vice-versa.

In [ ]:
import pulser

# Blockade radius from Rabi frequency
omega = 1  # rad/μs
rb = pulser.AnalogDevice.rydberg_blockade_radius(omega)  # μm
print(f"Rydberg blockade radius for Ω={omega} rad/μs: {rb} μm")

# Rabi frequency from Blockade radius
rb = 8
omega = pulser.AnalogDevice.rabi_from_blockade(rb)  # rad/μs
print(f"Rydberg blockade radius for Ω={omega} rad/μs: {rb} μm")

### Visualising interactions

The `Register.draw()` method includes options to plot the Rydberg blockade radius and identifiy interacting atoms. By specifying a value for `blockade_radius`,
- `draw_half_radius=True` draws circles with **half** the Rydberg blockade radius, so that when two circles overlap, the atoms are within a blockade radius of each other;
- `draw_graph=True` draws edges between the atoms within a blockade radius of each other, creating an approximate connectivity graph for the atoms in the system.

In [ ]:
from pulser import Register

# 4x3 triangular lattice with 6μm spacing
tri_reg = Register.triangular_lattice(
    rows=4, atoms_per_row=3, spacing=6.0, prefix="q"
)
# Draw the interactions for Rb=7 μm
tri_reg.draw(
    blockade_radius=7,  # μm
    draw_half_radius=True,  # Draws circles with radius Rb/2
    draw_graph=True,  # Draws edges between interacting atoms
)

## Tips for `Register` design

Choosing the best position for the atoms in a `Register` is generally a hard problem and depends heavily on the application. In this section, we provide some strategies that, while far from comprehensive, may help in the `Register` creation process in specfic cases.

### Think of the full Hamiltonian

When using a neutral-atom QPU to simulate a quantum many-body system, it is important to remember that the interaction Hamiltonian is but one part of the full Hamiltonian. In particular, the strength of the interaction terms must always be considered in relation to the driving Hamiltonian terms (i.e. $\Omega$ and $\delta$). 

Take the example of [AFM state preparation](tutorials/creating.nblink#Adiabatic-preparation-of-an-Anti-Ferromagnetic-State), where the interaction strength must be balanced with the appropriate value of $\delta>0$; without taking the full Hamiltonian into account, we could end up with:
- $\delta$ too low, which would not promote atoms to the $|r\rangle$ state, or
- $\delta$ too high, which would make all atoms go to the $|r\rangle$, regardless of their nearest neighbours being also in $|r\rangle$.

In these cases, it is only by first considering the full Hamiltonian that we are able to correctly design the register.

### Encode a cost function

Akin to a penalty term in a cost function, the interaction Hamiltonian makes some quantum states energetically less favourable. By appropriately adjusting the distances between atoms, the penalty of specific configurations can sometimes be replicated in the interaction Hamiltonian.

Examples where this approach is useful include:
- some instances of [QUBO](https://en.wikipedia.org/wiki/Quadratic_unconstrained_binary_optimization) problems,
- other optimization problems where the ground-state of the Hamiltonian encodes a minimizer of the cost function.


### Start from a connectivity graph

In some problems, we only care if some atoms interact while others don't, not the strength of the interactions. In these cases, the Rybderg blockade radius provides a useful approximation by allowing us to place interacting atoms well within a blockade radius of each other and non-interacting atoms well outside it. This approach is particularly straigthfoward when the connectivity graph can be represented as a [Unit-Disk graph](https://en.wikipedia.org/wiki/Unit_disk_graph). 

Examples where this approach is useful include:
- finding the Maximum Independent Set of a Unit-Disk graph,
- placing atoms for execution of multi-qubit gates.